In [1]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [2]:
import pandas
train = pandas.read_json(r'C:\Users\JYellin\re_1\tacred\data\json-enhanced\train.json')
dev = pandas.read_json(r'C:\Users\JYellin\re_1\tacred\data\json-enhanced\dev.json')
test = pandas.read_json(r'C:\Users\JYellin\re_1\tacred\data\json-enhanced\test.json')
data = pandas.concat([train,dev,test])

In [3]:
relations = test['relation'].unique().tolist()
relations.remove('no_relation')
avg_filtered_misses_by_relation =  {relation:0 for relation in relations}
avg_unfiltered_misses_by_relation = {relation:0 for relation in relations}


ucca_lengths = test['ucca_path_len'].unique().tolist()
avg_filtered_misses_by_ucca_length = {length:0 for length in ucca_lengths}
avg_unfiltered_misses_by_ucca_length = {length:0 for length in ucca_lengths}


ud_lengths = test['ud_path_len'].unique().tolist()
avg_filtered_misses_by_ud_length = {length:0 for length in ud_lengths}
avg_unfiltered_misses_by_ud_length = {length:0 for length in ud_lengths}



In [4]:
def get_gold(r):
    if not pandas.isnull(r['gold_x']):
        return r['gold_x']
    
    return r['gold_y']

for i in range(1,5):
    unfiltered_misses = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\analysis\filtering-by-path-length\comparing misses\ucca\comparison-{}\ucca-misses-nofilter.csv'.format(i)).rename(columns={'predicted':'no-filter-prediction'})
    filtered_misses = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\analysis\filtering-by-path-length\comparing misses\ucca\comparison-{}\ucca-misses-filter9.csv'.format(i)).rename(columns={'predicted':'filter-prediction'})
    
    merged_misses = filtered_misses.merge(unfiltered_misses, how='outer', on=['id'], indicator=True)
    merged_misses['gold'] = merged_misses.apply(lambda r : get_gold(r), axis=1)
    merged_misses.drop(columns=['gold_x', 'gold_y'], inplace=True)
    
    merged_misses = merged_misses.merge(data, how='left', on=['id'])
    
    #common_misses = merged_misses[merged_misses['_merge']=='both'].drop(columns=['_merge']) 
    filtered_only_misses = merged_misses[merged_misses['_merge']=='left_only'].drop(columns=['_merge','no-filter-prediction'])
    unfiltered_only_misses = merged_misses[merged_misses['_merge']=='right_only'].drop(columns=['_merge','filter-prediction'])
    
    
    # by gold ..
    filtered_only_misses_by_gold = filtered_only_misses.groupby(['gold'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['total'], ascending=[False]).reset_index(drop=True)
    filtered_only_misses_by_gold_dict = filtered_only_misses_by_gold.to_dict()
    filtered_lookup_index = dict((y,x) for x,y in filtered_only_misses_by_gold_dict['gold'].items())
    
    for relation in relations:
        if relation in filtered_lookup_index:
            lookup_index = filtered_lookup_index[relation]
            misses = filtered_only_misses_by_gold_dict['total'][lookup_index]
            avg_filtered_misses_by_relation[relation] = (avg_filtered_misses_by_relation[relation] * (i-1) + misses) / i

    unfiltered_only_misses_by_gold = unfiltered_only_misses.groupby(['gold'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['total'], ascending=[False]).reset_index(drop=True)
    unfiltered_only_misses_by_gold_dict = unfiltered_only_misses_by_gold.to_dict()
    unfiltered_lookup_index = dict((y,x) for x,y in unfiltered_only_misses_by_gold_dict['gold'].items())
    
    for relation in relations:
        if relation in unfiltered_lookup_index:
            lookup_index = unfiltered_lookup_index[relation]
            misses = unfiltered_only_misses_by_gold_dict['total'][lookup_index]
            avg_unfiltered_misses_by_relation[relation] = (avg_unfiltered_misses_by_relation[relation] * (i-1) + misses) / i
            

    # by ucca path length ..
    filtered_only_misses_by_ucca_len = filtered_only_misses.groupby(['ucca_path_len'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['total'], ascending=[False]).reset_index(drop=True)
    filtered_only_misses_by_ucca_len_dict = filtered_only_misses_by_ucca_len.to_dict()
    filtered_lookup_index = dict((y,x) for x,y in filtered_only_misses_by_ucca_len['ucca_path_len'].items())
    
    for ucca_length in ucca_lengths:
        if ucca_length in filtered_lookup_index:
            lookup_index = filtered_lookup_index[ucca_length]
            misses = filtered_only_misses_by_ucca_len_dict['total'][lookup_index]
            avg_filtered_misses_by_ucca_length[ucca_length] = (avg_filtered_misses_by_ucca_length[ucca_length] * (i-1) + misses) / i
            
            
    unfiltered_only_misses_by_ucca_len = unfiltered_only_misses.groupby(['ucca_path_len'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['total'], ascending=[False]).reset_index(drop=True)
    unfiltered_only_misses_by_ucca_len_dict = unfiltered_only_misses_by_ucca_len.to_dict()
    unfiltered_lookup_index = dict((y,x) for x,y in unfiltered_only_misses_by_ucca_len['ucca_path_len'].items())
    
    for ucca_length in ucca_lengths:
        if ucca_length in unfiltered_lookup_index:
            lookup_index = unfiltered_lookup_index[ucca_length]
            misses = unfiltered_only_misses_by_ucca_len_dict['total'][lookup_index]
            avg_unfiltered_misses_by_ucca_length[ucca_length] = (avg_unfiltered_misses_by_ucca_length[ucca_length] * (i-1) + misses) / i
    
    
    
    # by ud path length ..
    filtered_only_misses_by_ud_len = filtered_only_misses.groupby(['ud_path_len'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['total'], ascending=[False]).reset_index(drop=True)
    filtered_only_misses_by_ud_len_dict = filtered_only_misses_by_ud_len.to_dict()
    filtered_lookup_index = dict((y,x) for x,y in filtered_only_misses_by_ud_len['ud_path_len'].items())
    
    for ud_length in ud_lengths:
        if ud_length in filtered_lookup_index:
            lookup_index = filtered_lookup_index[ud_length]
            misses = filtered_only_misses_by_ud_len_dict['total'][lookup_index]
            avg_filtered_misses_by_ud_length[ud_length] = (avg_filtered_misses_by_ud_length[ud_length] * (i-1) + misses) / i
            
            
    unfiltered_only_misses_by_ud_len = unfiltered_only_misses.groupby(['ud_path_len'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['total'], ascending=[False]).reset_index(drop=True)
    unfiltered_only_misses_by_ud_len_dict = unfiltered_only_misses_by_ud_len.to_dict()
    unfiltered_lookup_index = dict((y,x) for x,y in unfiltered_only_misses_by_ud_len['ud_path_len'].items())
    
    for ud_length in ud_lengths:
        if ud_length in unfiltered_lookup_index:
            lookup_index = unfiltered_lookup_index[ud_length]
            misses = unfiltered_only_misses_by_ud_len_dict['total'][lookup_index]
            avg_unfiltered_misses_by_ud_length[ud_length] = (avg_unfiltered_misses_by_ud_length[ud_length] * (i-1) + misses) / i



In [5]:
# sort relations ..
relations = sorted(avg_filtered_misses_by_relation,key=avg_filtered_misses_by_relation.get, reverse=True)


scatter1 = go.Scatter(
    x=[relation for relation in relations], 
    y=[avg_filtered_misses_by_relation[relation] for relation in relations],
    mode='lines',
    name='filtered',
    marker = dict(color='#BC80BD')
    
)

scatter2= go.Scatter(
    x=[relation for relation in relations], 
    y=[avg_unfiltered_misses_by_relation[relation] for relation in relations],
    mode='lines',
    name='unfiltered',
    marker = dict(color='#CCEBC5')

)


layout = go.Layout(
    title='Miss Per Relation',
    xaxis_title="Path Length",
    yaxis_title="Number of Misses",        
    barmode='overlay',
    width=1050,
    xaxis = go.layout.XAxis(
        tickangle = 45,
        automargin = True
    )
)

fig = go.Figure(data=[scatter1,scatter2], layout=layout)


fig.show()

In [6]:

scatter1 = go.Scatter(
    x=[length for length in range(-1,10)], 
    y=[avg_filtered_misses_by_ucca_length[length] if length in avg_filtered_misses_by_ucca_length else 0 for length in range(-1,10)],
    mode='lines',
    name='filtered',
    marker = dict(color='#BC80BD')
    
)

scatter2= go.Scatter(
    x=[length for length in range(-1,10)], 
    y=[avg_unfiltered_misses_by_ucca_length[length] if length in avg_filtered_misses_by_ucca_length else 0 for length in range(-1,10)],
    mode='lines',
    name='unfiltered',
    marker = dict(color='#CCEBC5')

)


layout = go.Layout(
    title='Miss Per UCCA path length',
    xaxis_title="Path Length",
    yaxis_title="Number of Misses",        
    barmode='overlay',
    width=1050,
    xaxis = go.layout.XAxis(
        tickangle = 45,
        automargin = True
    )
)

fig = go.Figure(data=[scatter1,scatter2], layout=layout)


fig.show()

In [7]:

scatter1 = go.Scatter(
    x=[length for length in range(-1,10)], 
    y=[avg_filtered_misses_by_ud_length[length] if length in avg_filtered_misses_by_ud_length else 0 for length in range(-1,10)],
    mode='lines',
    name='filtered',
    marker = dict(color='#BC80BD')
    
)

scatter2= go.Scatter(
    x=[length for length in range(-1,10)], 
    y=[avg_unfiltered_misses_by_ud_length[length] if length in avg_filtered_misses_by_ud_length else 0 for length in range(-1,10)],
    mode='lines',
    name='unfiltered',
    marker = dict(color='#CCEBC5')

)


layout = go.Layout(
    title='Miss Per UD path length',
    xaxis_title="Path Length",
    yaxis_title="Number of Misses",        
    barmode='overlay',
    width=1050,
    xaxis = go.layout.XAxis(
        tickangle = 45,
        automargin = True
    )
)

fig = go.Figure(data=[scatter1,scatter2], layout=layout)


fig.show()